# Bodybuilding WFPB Diet
Author: Simon Thornewill von Essen

Date: 2019-05-02

## Overview

In this jupyter notebook I want to do analysis on the data that I have collected on various foods that I buy on a daily basis and see what the optimal mix of these foods are to meet my daily requirements for bodybuilding. (For more information, see the README.)

I want to do this analysis in a number of steps:

1. Import relevant data and packages
2. Quickly clean the data so that it is ready for analysis
    * Should be relatively quick because I had to create this data by hand so it is mostly in the format I want already.
3. Do a quick exploratory data analysis to get some understanding of the nutrients in my food
4. Use linear programming to determine the optimal mix of foods while minimising the total cost

All of the values used for nutrients are from cronometer.com and numbers for EDAs and other such targets can be found in the README.

## Import Data and packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from pulp import *

plt.style.use("seaborn")

%matplotlib inline

In [2]:
# Import foods data
df = pd.read_csv("./dat/foods.csv")

df.head()

,food_item,pcs,pcs_g,pcs_ml,cost_eur,energy_kcal,alcohol_g,caffeine_mg,water_g,carbs_g,...,calcium_mg,copper_mg,iron_mg,magnesium_mg,manganese_mg,phosphorus_mg,potassium_mg,selenium_ug,sodium_mg,zinc_mg
0,oat_soy_mlk,NaN,NaN,100.0,0.14,45.0,NaN,NaN,85.0,NaN,...,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,NaN
1,soy_mlk,NaN,NaN,100.0,0.14,46.0,NaN,NaN,85.0,NaN,...,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,NaN
2,lentils,NaN,100.0,NaN,0.72,358.0,NaN,NaN,7.8,63.1,...,48.0,1.3,7.4,5.9,1.7,294.0,668.0,NaN,NaN,3.6
3,banana,NaN,100.0,NaN,0.20,89.0,NaN,NaN,74.9,22.8,...,5.0,0.1,0.3,27.0,0.3,22.0,358.0,1.0,1.0,0.2
4,strawberry,NaN,100.0,NaN,0.16,32.0,NaN,NaN,91.0,7.7,...,16.0,NaN,0.4,13.0,0.4,24.0,153.0,0.4,1.0,0.1


In [3]:
# Import RDA data
df_RDA = pd.read_csv("./dat/requirements.csv")

df_RDA

,operation,energy_kcal,alcohol_g,caffeine_mg,water_g,carbs_g,fiber_g,starch_g,sugars_g,fat_g,...,calcium_mg,copper_mg,iron_mg,magnesium_mg,manganese_mg,phosphorus_mg,potassium_mg,selenium_ug,sodium_mg,zinc_mg
0,max,2641.0,0.0,0.0,3700.0,130.0,38.0,0.0,0.0,102.7,...,1000.0,0.9,8.0,400.0,2.3,700.0,3400.0,55.0,1500.0,11.0
1,min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Clean Data
You can see above a list of foods, cost, quantity, and their associated nutritional information. 

One thing I did deliberately while building this dataset by hand was leaving Null values. This is because I intended to fill them with `0`s later using pandas's functionality to do such things.

In [4]:
for frame in [df, df_RDA]:
    frame.fillna(value=0, inplace=True)

df.head()

,food_item,pcs,pcs_g,pcs_ml,cost_eur,energy_kcal,alcohol_g,caffeine_mg,water_g,carbs_g,...,calcium_mg,copper_mg,iron_mg,magnesium_mg,manganese_mg,phosphorus_mg,potassium_mg,selenium_ug,sodium_mg,zinc_mg
0,oat_soy_mlk,0.0,0.0,100.0,0.14,45.0,0.0,0.0,85.0,0.0,...,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0
1,soy_mlk,0.0,0.0,100.0,0.14,46.0,0.0,0.0,85.0,0.0,...,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,0.0
2,lentils,0.0,100.0,0.0,0.72,358.0,0.0,0.0,7.8,63.1,...,48.0,1.3,7.4,5.9,1.7,294.0,668.0,0.0,0.0,3.6
3,banana,0.0,100.0,0.0,0.20,89.0,0.0,0.0,74.9,22.8,...,5.0,0.1,0.3,27.0,0.3,22.0,358.0,1.0,1.0,0.2
4,strawberry,0.0,100.0,0.0,0.16,32.0,0.0,0.0,91.0,7.7,...,16.0,0.0,0.4,13.0,0.4,24.0,153.0,0.4,1.0,0.1


In [5]:
df_RDA.head()

,operation,energy_kcal,alcohol_g,caffeine_mg,water_g,carbs_g,fiber_g,starch_g,sugars_g,fat_g,...,calcium_mg,copper_mg,iron_mg,magnesium_mg,manganese_mg,phosphorus_mg,potassium_mg,selenium_ug,sodium_mg,zinc_mg
0,max,2641.0,0.0,0.0,3700.0,130.0,38.0,0.0,0.0,102.7,...,1000.0,0.9,8.0,400.0,2.3,700.0,3400.0,55.0,1500.0,11.0
1,min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now that the null values have been filled, I want to drop columns that have only `0` values in them. Columns like alcohol and caffiene have been completely unused for example.

In [6]:
# Find columns that have a sum of 0, i.e. completely unused
df.sum() 

food_item            oat_soy_mlksoy_mlklentilsbananastrawberrypeanu...
pcs                                                                  2
pcs_g                                                             2546
pcs_ml                                                             200
cost_eur                                                         13.23
energy_kcal                                                     4630.8
alcohol_g                                                            0
caffeine_mg                                                          0
water_g                                                         1827.5
carbs_g                                                          460.4
fiber_g                                                          178.2
starch_g                                                         142.3
sugars_g                                                         113.8
fat_g                                                            251.4
monoun

In [7]:
# Find only food with vitamin D
df.query("vit_d_IU > 0").food_item

14    champignons
Name: food_item, dtype: object

We can see in the columns above that the only other unused column is cholesterol, and so I'll drop that as well as the other two.

The only other two columns are vitamin B-12, which I am not concerned about since I take a suppliment and vitamin-D, which can only be found in mushrooms.

So I'll drop these columns

In [8]:
df.drop(labels=["cholesterol_mg", "vit_d_IU", "alcohol_g", "caffeine_mg"],
       axis=1,
       inplace = True)

df.head()

,food_item,pcs,pcs_g,pcs_ml,cost_eur,energy_kcal,water_g,carbs_g,fiber_g,starch_g,...,calcium_mg,copper_mg,iron_mg,magnesium_mg,manganese_mg,phosphorus_mg,potassium_mg,selenium_ug,sodium_mg,zinc_mg
0,oat_soy_mlk,0.0,0.0,100.0,0.14,45.0,85.0,0.0,0.0,0.0,...,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0
1,soy_mlk,0.0,0.0,100.0,0.14,46.0,85.0,0.0,0.0,0.0,...,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,0.0
2,lentils,0.0,100.0,0.0,0.72,358.0,7.8,63.1,63.1,47.6,...,48.0,1.3,7.4,5.9,1.7,294.0,668.0,0.0,0.0,3.6
3,banana,0.0,100.0,0.0,0.20,89.0,74.9,22.8,2.6,5.4,...,5.0,0.1,0.3,27.0,0.3,22.0,358.0,1.0,1.0,0.2
4,strawberry,0.0,100.0,0.0,0.16,32.0,91.0,7.7,2.0,0.0,...,16.0,0.0,0.4,13.0,0.4,24.0,153.0,0.4,1.0,0.1


I suppose the only difficulty left is that when doing the linear programming that there is no one clear column that can be used to tweak the amounts of values. Instead, we have three columns that can be used towards this purpose, `pcs`, `pcs_g` and `pcs_ml`.

these should be combined into a single column for use in linear programming.

In [9]:
# Create dataframe
df_amount = df[["pcs", "pcs_g", "pcs_ml"]]

df_amount.head()

,pcs,pcs_g,pcs_ml
0,0.0,0.0,100.0
1,0.0,0.0,100.0
2,0.0,100.0,0.0
3,0.0,100.0,0.0
4,0.0,100.0,0.0


In [10]:
# Replace 0 values with NaN
df_amount.replace(to_replace=0,
                 value=np.NaN,
                 inplace = True);

# Coalesce columns
df["amount"] =  df_amount["pcs_g"].combine_first(df_amount["pcs_ml"].combine_first(df_amount["pcs"]))

# Reordering columns
df = df[['food_item','amount', 'pcs', 'pcs_g', 'pcs_ml', 'cost_eur', 'energy_kcal',
       'water_g', 'carbs_g', 'fiber_g', 'starch_g', 'sugars_g', 'fat_g',
       'monounsaturated_g', 'polyunsaturated_g', 'omega_3_g', 'omega_6_g',
       'saturated_g', 'trans_fats_g', 'protein_g', 'vit_b1_mg', 'vit_b2_mg',
       'vit_b3_mg', 'vit_b5_mg', 'vit_b6_mg', 'vit_b12_ug', 'folate_ug',
       'vit_a_IU', 'vit_c_mg', 'vit_e_mg', 'vit_k_ug', 'calcium_mg',
       'copper_mg', 'iron_mg', 'magnesium_mg', 'manganese_mg', 'phosphorus_mg',
       'potassium_mg', 'selenium_ug', 'sodium_mg', 'zinc_mg']]

df.head()

/Users/simonthornewillvonessen/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


,food_item,amount,pcs,pcs_g,pcs_ml,cost_eur,energy_kcal,water_g,carbs_g,fiber_g,...,calcium_mg,copper_mg,iron_mg,magnesium_mg,manganese_mg,phosphorus_mg,potassium_mg,selenium_ug,sodium_mg,zinc_mg
0,oat_soy_mlk,100.0,0.0,0.0,100.0,0.14,45.0,85.0,0.0,0.0,...,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0
1,soy_mlk,100.0,0.0,0.0,100.0,0.14,46.0,85.0,0.0,0.0,...,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,0.0
2,lentils,100.0,0.0,100.0,0.0,0.72,358.0,7.8,63.1,63.1,...,48.0,1.3,7.4,5.9,1.7,294.0,668.0,0.0,0.0,3.6
3,banana,100.0,0.0,100.0,0.0,0.20,89.0,74.9,22.8,2.6,...,5.0,0.1,0.3,27.0,0.3,22.0,358.0,1.0,1.0,0.2
4,strawberry,100.0,0.0,100.0,0.0,0.16,32.0,91.0,7.7,2.0,...,16.0,0.0,0.4,13.0,0.4,24.0,153.0,0.4,1.0,0.1


## Exploratory Data Analysis

Now that the data cleaning has been done, I want to take a look at these foods in terms of which one is highest in the nutrients that I am interested in. 

I'll take the columns of my dataframe and break them up into `protein`, `carbs`, `lipids` and `minerals`, since these were the major classes that could be found on cronometer.

Note that I didn't break down proteins because most plant based protein sources are either complete proteins are mostly complete. Specific proteins is not something you need to worry about when it comes to creating an optimal diet and so it is not of interest to me. 

In [11]:
df.sort_values(by=["fiber_g"], ascending=False).head(5)[["food_item", "fiber_g"]].values[:, 0]

array(['lentils', 'chia_seeds', 'avocado', 'granny_smith', 'walnuts'],
      dtype=object)

In [12]:
"""
# Next, I'll create a general plot that I will generalise into a function later
plot_nutrient = "fiber_g"

for classification in vitamins.keys():
    for plot_nutrient in vitamins[classification]:
        
        # Create sorted array
        X_1 = df.sort_values(by=[plot_nutrient], 
                           ascending=False).head(5)[["food_item", plot_nutrient]].sort_values(plot_nutrient).values

        # Build plot
        fig, ax = plt.subplots(figsize=(10, 6), facecolor='white', edgecolor='k')
        ax.barh(X_1[:, 0],
                X_1[:, 1])
        plt.xlabel(plot_nutrient)
        plt.ylabel("Food Item")
        plt.title("Amounts of nutrient in food item")
        # plt.savefig("./img/plot_{}".format(plot_nutrient))
        plt.close(fig)
"""

'\n# Next, I\'ll create a general plot that I will generalise into a function later\nplot_nutrient = "fiber_g"\n\nfor classification in vitamins.keys():\n    for plot_nutrient in vitamins[classification]:\n        \n        # Create sorted array\n        X_1 = df.sort_values(by=[plot_nutrient], \n                           ascending=False).head(5)[["food_item", plot_nutrient]].sort_values(plot_nutrient).values\n\n        # Build plot\n        fig, ax = plt.subplots(figsize=(10, 6), facecolor=\'white\', edgecolor=\'k\')\n        ax.barh(X_1[:, 0],\n                X_1[:, 1])\n        plt.xlabel(plot_nutrient)\n        plt.ylabel("Food Item")\n        plt.title("Amounts of nutrient in food item")\n        # plt.savefig("./img/plot_{}".format(plot_nutrient))\n        plt.close(fig)\n'

## Linear Program

To build a Linear Program (LP) you need to minimise or maximise an objective function subject to some constraints.

$$max. c^TX$$
$$s.t. Ax = b$$
$$ x \geq 0$$

Where $c^TX$ is the objective function. In this case, X are the proportions of foods bought and c is a vector of the costs for each unit of food item. 

However, if we want to maximise something then why not just take an infinite amount of each food item or if we want to minimise the O.F. then why not just take nothing? That's where the constraints $Ax = b$ come in. In this situation, we want a minimim dose for certain foods, or a maximum of nutrients that we dont want in our diet such as saturated/trans fat. 

However, we also want the added constraint that any amounts of food that we choose is greater than 0. We don't want -ve quantities of food, which represents something like selling food in this situation.

And voila, that's our whole linear program. 

In one of my [other notebooks](https://github.com/SThornewillvE/Pet-Project---Python-Linear-Solver/blob/master/solver.ipynb) I created a basic guide for determining an ideal diet but its not using any real data. I want to change that here. 

For this, we'll use the `pulp` library, but other libraries also exist.

### Step 1 - Initialise Program

First, we need to create our problem, in PULP we'll name and say whether or want to minimise or maximise our objective function.

In [13]:
# Define problem
problem = pulp.LpProblem('wfpbDiet', pulp.LpMinimize)

### Step 2 - Create Objective function

Now that our linear program has been established, we want to establish our (decision) variables. We can decide if we want to have full integers, continuous variables, or binary variables using `Integer`, `Binary` or `Continuous` respectively.

For our purposes, we don't care if we have whole numbers for our ingredients. We also don't care about binary variables so we'll choose to have continuous variables.

The difficulty is that we want to define a lot of variables which can be a real pain. So we'll define them all in a dictionary.

In [14]:
# Create foods vector
foods =  df.food_item.values

In [15]:
# create LP variable dictionary
lp_vars = {}

for i, food in enumerate(foods):
    lp_vars["x_{}".format(i)] = LpVariable(food, 0, None, LpContinuous)

In [16]:
# Initialise list
xvars = []

# Fill list with values
for i in range(len(lp_vars.keys())):
    xvars.append(lp_vars["x_{}".format(i)])

# Convert list to array
xvars = np.array(xvars)

Now that we have our variables defined, we also want to be able to access this as a vector so that its easy to manipulate.

With our niffty xvars vector in hand we can start worryig about the costs component of our objective function.

With our two vectors defined we can use the dot product of these vectors to create our objective function.

In [17]:
# Create cost vector
costvars =  df.cost_eur.values

problem +=  xvars.dot(costvars)

This means that we have successfully constructed our objective function that we want to minimise as defined in our problem from the last step.


### Step 3 - Create constraints

With our objective function we created a mapping as to how much each food item costs. However, we are still missing important information such as how many nutrients each foodstuff has and what we are looking for in terms of nutrition. This is why we need to think about our constraints.

The basic format that we're looking for is to have lists of nutrient values for each nutrient in order. We then want to say that the dot product of these values and the food stuffs needs to be above or below a certain value. These values are something I have defined already in another dataframe

What we want is of this kind of form...

$$problem \leftarrow xvars.dot(nutrients) \geq RDA$$

We already have our `xvars`, so for each one we want to have a list of a specific nutrient. 

In [18]:
# Create vitamin list
vitamin_list = df.columns.tolist()

# Remove columns that aren't vitamins
for i in ["food_item", "amount", "pcs", "pcs_g", "pcs_ml", "cost_eur", "starch_g", "sugars_g", "trans_fats_g", "vit_b12_ug"]:
    vitamin_list.remove(i)

In [19]:
# Create constraints
for vit in vitamin_list:
    if df_RDA.query("operation == 'max'")[vit][0] > 0:
        problem += xvars.dot(df[vit].values) >= df_RDA.query("operation == 'max'")[vit][0], 'min {}'.format(vit)
    else:
        problem += xvars.dot(df[vit].values) <= df_RDA.query("operation == 'min'")[vit][1], 'min {}'.format(vit)

Success! This means that our problem is entirely defined and we can go on to solving our problem!

### Part 4 - Solve problem

Solving the problem is relatively easy, the hard part is setting up the problem.

In [21]:
# Solve Problem
if problem.solve():  # Note that prob.solve() will be 1 if the solution converges else 0
    print("Status: ", LpStatus[problem.status])
    for v in problem.variables():
        print(v.name, "=", v.varValue)
else: print("No Solution Found.")

Status:  Optimal
avocado = 0.0
baby_spinach = 7.4195883
banana = 0.0
bell_pepper = 0.0
broccoli = 0.0
brown_rice = 0.0
canned_chick_bnz = 0.0
canned_corn = 0.0
canned_kidney_bnz = 0.0
carrots = 0.0
champignons = 324.93345
chia_seeds = 0.04820456
firm_tofu = 0.0
flour_tortilla = 0.0
froz_blueberries = 0.0
froz_rasberries = 0.0
froz_strawberries = 0.0
granny_smith = 0.0
lentils = 0.0
oat_soy_mlk = 0.0
onions = 0.0
peanut_butter = 0.015024305
pumpkin_seeds = 0.0
quinoa = 0.0
soy_mlk = 0.0
strawberry = 0.0
walnuts = 0.0


Well, that was messed up.

Turns out that the most optimal vegan body building diet is a mixture of a LOT of mushrooms, spinach, peanut butter and chia seeds. Even if it is optimal, humans would get sick of this very quickly. 

This is a problem that people have run into often with linear programming so it's not that this output is wrong. I just thought that I would be able to solve a problem that doesn't necessarily apply to me. The basic issue is that this optimisation is a little too strong and will only allow you to create one "meal" with foods that don't even necessarily work together.

There was one issue that I ran into where my program wasn't "feasible", this is because I wanted to meet a certain recommendation for b12 but ofcourse, I don't have any foods that can meet that constraint and so removing it fixed that problem.